#### Introduction

This lesson explores three different methods of sending ETH from one account to another: `transfer`, `send`, and `call`. We will understand their differences, how each one works, and when to use one instead of another.

#### Transfer

The `transfer` function is the simplest way to send Ether to a recipient address:

```solidity
payable(msg.sender).transfer(amount); // the current contract sends the Ether amount to the msg.sender
```

It's necessary to convert the recipient address to a **payable** address to allow it to receive Ether. This can be done by wrapping `msg.sender` with the `payable` keyword.

However, `transfer` has significant limitation. It can only use up to 2300 gas and it reverts any transaction that exceeds this gas limit, as illustrated by [Solidity by Example](https://solidity-by-example.org/sending-ether/).

#### Send

The `send` function is similar to `transfer`, but it differs in its behavior:

```solidity
bool success = payable(msg.sender).send(address(this).balance);
require(success, "Send failed");
```

Like `transfer`, `send` also has a gas limit of 2300. If the gas limit is reached, it will not revert the transaction but return a boolean value (`true` or `false`) to indicate the success or failure of the transaction. It is the developer's responsibility to handle failure correctly, and it's good practice to trigger a **revert** condition if the `send` returns `false`.

#### Call

The `call` function is flexible and powerful. It can be used to call any function **without requiring its ABI**. It does not have a gas limit, and like `send`, it returns a boolean value instead of reverting like `transfer`.

```solidity 
(bool success, ) = payable(msg.sender).call{value: address(this).balance}("");
require(success, "Call failed");
```

To send funds using the `call` function, we convert the address of the receiver to `payable` and add the value inside curly brackets before the parameters passed.

The `call` function returns two variables: a boolean for success or failure, and a byte object which stores returned data if any.

> 👀❗ IMPORTANT  
> `call` is the recommended way of sending and receiving Ethereum or other blockchain native tokens.

#### Conclusion 

In conclusion, *transfer*, *send*, and *call* are three unique methods for transferring Ether in Solidity. They vary in their syntax, behavior, and gas limits, each offering distinct advantages and drawbacks.

#### Test yourself

1. What are the primary differences between *transfer*, *send*, and *call* when transferring Ether?
   * Transfer
      * Gas forwarded: Fixed 2300 gas
      * Error handling: Reverts on failure
      * Recommended for: Simple transfers
      * Security: Safe against reentrancy (because of the 2300 gas limit)
   * Send
      * Gas forwarded: Fixed 2300 gas
      * Error handling: Returns `false` on failure (does not revert)
      * Recommendd for: Rarely used today (mostly legacy)
   * Call
      * Gas forwarded: Forwards all remaining gas by default (customizable)
      * Error handling: Returns a success flag and optional data
      * Recommended for: Preferred method for sending ETH since Solidity 0.6+
      * Security: Must be protected with reentrancy guards

2. Why is it necessary to convert an address to `payable` type before sending Ether to it?
   * The `address` type is used for referencing any address on the blockchain but it **does not include functions to transfer Ether**. To make a transfer, Solidity requires that you **explicitly signal your intent** by converting it to `address payable`.

3. Implement a function `callAmountTo` using `call` to send Ether from the contract to an address provided as an argument. Ensure the function handles failures appropriately.

```solidity
function callAmountTo(address recipient) public {
    (bool success, ) = payable(recipient).call{value: address(msg.value).balance}("");
    
    require(success, "Call failed");
}
```